# AutoGraph

In this tutorial, we will introduce how to convert python function into executable
Tensorflow graphs.

In [1]:
import inspect
import time
import numpy as np
import tensorflow as tf
from pprint import pprint

In [2]:
print(tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)
true_weights = tf.constant(list(range(5)), dtype=tf.float32)[:, tf.newaxis]
x = tf.constant(tf.random.uniform((32, 5)), dtype=tf.float32)
y = tf.constant(x @ true_weights, dtype=tf.float32)

2.4.1


## AutoGraph

A computational graph contains two things: computation and data. Tensorflow graph `tf.Graph` is the computational graph made from operations as computation units and tensors as data units. Tensorflow has many optimizations around graphs, so executing in graph mode result in better utilization of distributed computing. Instead of writing complicated graph mode code, Tensorflow 2 provides a tool AutoGraph to automatically analyze and convert python code into graph code which can then be traced to create a graph with Function.

The following python function involves relatively simple math operations on tensors, so it is pretty much graph ready. Let's see if `tf.autograph` would do anything interesting to it.

In [6]:
def f(a, b, power=2, d=3):
    return tf.pow(a, power) + d * b

converted_f = tf.autograph.to_graph(f)
print(inspect.getsource(converted_f))

        def tf__f(a, b, power=None, d=None):
            with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
                do_return = False
                retval_ = ag__.UndefinedReturnValue()
                try:
                    do_return = True
                    retval_ = (ag__.converted_call(ag__.ld(tf).pow, (ag__.ld(a), ag__.ld(power)), None, fscope) + (ag__.ld(d) * ag__.ld(b)))
                except:
                    do_return = False
                    raise
                return fscope.ret(retval_, do_return)



In [7]:
print(inspect.getsource(f))

def f(a, b, power=2, d=3):
    return tf.pow(a, power) + d * b



Let's move on to another python function with a bit graph unfriendly construct.

In [8]:
def cube(x):
    o = x
    for _ in range(2):
        o *= x
    return o

converted_cube = tf.autograph.to_graph(cube)
print(inspect.getsource(converted_cube))

        def tf__cube(x):
            with ag__.FunctionScope('cube', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
                do_return = False
                retval_ = ag__.UndefinedReturnValue()
                o = ag__.ld(x)

                def get_state():
                    return (o,)

                def set_state(vars_):
                    nonlocal o
                    (o,) = vars_

                def loop_body(itr):
                    nonlocal o
                    _ = itr
                    o = ag__.ld(o)
                    o *= x
                _ = ag__.Undefined('_')
                ag__.for_stmt(ag__.converted_call(ag__.ld(range), (2,), None, fscope), None, loop_body, get_state, set_state, ('o',), {'iterate_names': '_'})
                try:
                    do_return = True
                    retval_ = ag__.ld(o)
                except:
                    do_retu

We see that body of the for loop is converted into a *loop_body* function, which is invoked by *autograph.for_stmt*. This `for_stmt` operation is, in a sense, overloads the for statement. The purpose of this transformation is to make the code into a functional style so that it can be executed in graph.

In [9]:
def g(x):
    if tf.reduce_any(x < 0):
        return tf.square(x)
    return x

converted_g = tf.autograph.to_graph(g)
print(inspect.getsource(converted_g))

        def tf__g(x):
            with ag__.FunctionScope('g', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
                do_return = False
                retval_ = ag__.UndefinedReturnValue()

                def get_state():
                    return (do_return, retval_)

                def set_state(vars_):
                    nonlocal do_return, retval_
                    (do_return, retval_) = vars_

                def if_body():
                    nonlocal do_return, retval_
                    try:
                        do_return = True
                        retval_ = ag__.converted_call(ag__.ld(tf).square, (ag__.ld(x),), None, fscope)
                    except:
                        do_return = False
                        raise

                def else_body():
                    nonlocal do_return, retval_
                    try:
                        do_return = Tr

The big idea about AutoGraph is that it translates the python code we wrote into a style that can be traced to create Tensorflow graphs. During the transformation, great efforts were made to rewrite the data-dependent conditionals and control flows, as these statements can't be straight forward overloaded by Tensorflow operations. There are a lot more about AutoGraph, the interested reader can refer to the [paper](https://arxiv.org/abs/1810.08061) for more details.

## Functions

Once the code is graph friendly, we can trace the operations in the code to create a graph. The created graph then gets wrapped up in a `ConcreteFunction` object so that we can execute computations backed in graph mode with it. 

Lets walkthrough how these works. First we provide Tensorflow our graph friendly code to
create a `Function` object.

In [12]:
tf_func_f = tf.function(autograph=False)(f)
tf_func_g = tf.function(autograph=False)(converted_g)
tf_func_g2 = tf.function(autograph=True)(g)
print(tf_func_f.python_function is f)
print(tf_func_g.python_function is converted_g)
print(tf_func_g2.python_function is g)

True
True
True


As of now, there is no graph created yet. We only attached our function to this Functionobject. Notice that we specified `autograph=False` when constructing the Function in the first two cases, because we know both f and converted_g are graph friendly. But good old g is not, so we need to turn on autograph for it. In fact,
`tf.function(autograph=False)(tf.autograph.to_graph(g))` is roughly equivlent to `tf.function(autograph=True)(g)`.

Note that we can create Function with `tf.function(autograph=False)(g)` this will
succeed without error. But we won't be able to create any graphs with this Function in the
next step.

The next step is to provide Tensorflow a signature, i.e. a description of inputs, allowing it
to create a graph by tracing how the input tensors would flow through the operations in
the graph and record the graph in a callable object.

In [13]:
concrete_g = tf_func_g.get_concrete_function(x=tf.TensorSpec(shape=[3], dtype=tf.float32))
print(concrete_g)

ConcreteFunction tf__g(x)
  Args:
    x: float32 Tensor, shape=(3,)
  Returns:
    float32 Tensor, shape=(3,)


We can use this concrete function directly as if it is a operation shipped with Tensorflow,
or we can call the Function object, which will look up the concrete function and use it.
Either way, the computation will be executed with the created graph.

In [14]:
pprint(concrete_g(tf.constant([-1, 1, -2], dtype=tf.float32)))
pprint(tf_func_g(tf.constant([-1, 1, -2], dtype=tf.float32)))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 4.], dtype=float32)>
<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 4.], dtype=float32)>


The Function object is like a graph factory. When detailed input specifications were
provided, it uses the graph code as receipt to create new graphs. When asked with an
known specifications, it will dig up the graph in the storage and serve it. When called with
an unknown signature, it will trigger the creation of the concrete function first. Let's try to
make a bunch graphs.

In [15]:
concrete_f = tf_func_f.get_concrete_function(a=tf.TensorSpec(shape=[1], dtype=tf.float32), b=tf.TensorSpec(shape=[1], dtype=tf.float32))
print(concrete_f)
pprint(concrete_f(tf.constant(1.), tf.constant(2.)))
pprint(tf_func_f(1., 2.))
pprint(tf_func_f(a=tf.constant(1., dtype=tf.float32), b=2, power=2.))
pprint(tf_func_f(a=tf.constant(1., dtype=tf.float32), b=2., d=3))
pprint(tf_func_f(a=tf.constant(1., dtype=tf.float32), b=2., d=3., power=3.))

ConcreteFunction f(a, b, power=2, d=3)
  Args:
    a: float32 Tensor, shape=(1,)
    b: float32 Tensor, shape=(1,)
  Returns:
    float32 Tensor, shape=(1,)
<tf.Tensor: shape=(), dtype=float32, numpy=7.0>
<tf.Tensor: shape=(), dtype=float32, numpy=7.0>
<tf.Tensor: shape=(), dtype=float32, numpy=7.0>
<tf.Tensor: shape=(), dtype=float32, numpy=7.0>
<tf.Tensor: shape=(), dtype=float32, numpy=7.0>


In [16]:
## Print out the number of graphs we have created above
print(tf_func_f._get_tracing_count())

4


In [17]:
### ??
for i, f in enumerate(tf_func_f._list_all_concrete_functions_for_serialization()):
    print(i, f.structured_input_signature)

0 ((TensorSpec(shape=(1,), dtype=tf.float32, name='a'), TensorSpec(shape=(1,), dtype=tf.float32, name='b'), 2, 3), {})
1 ((1.0, 2.0, 2, 3), {})
2 ((TensorSpec(shape=(), dtype=tf.float32, name='a'), 2, 2.0, 3), {})
3 ((TensorSpec(shape=(), dtype=tf.float32, name='a'), 2.0, 3.0, 3.0), {})


Lastly, `tf.function` is also available as a decorator, which makes life easier. The
following two are equivalent

In [5]:
@tf.function(autograph=False)
def square(x):
    return x * x

square = tf.function(autograph=False)(square)

In [6]:
print(inspect.getsource(square))

@tf.function(autograph=False)
def square(x):
    return x * x



## Linear Regression Revisited

**Normal mode**

In [8]:
# ground truth
true_weights = tf.constant(list(range(5)), dtype=tf.float32)[:, tf.newaxis]

# some random training data
x = tf.constant(tf.random.uniform((32, 5)), dtype=tf.float32)
y = tf.constant(x @ true_weights, dtype=tf.float32)

t0 = time.time()

weights = tf.Variable(tf.random.uniform((5, 1)), dtype=tf.float32)
for iteration in range(1001):
    with tf.GradientTape() as tape:
        y_hat = tf.linalg.matmul(x, weights)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    if not (iteration % 200):
        print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))
    gradients = tape.gradient(loss, weights)
    weights.assign_add(-0.05 * gradients)

pprint(weights)
print('time took: {} seconds'.format(time.time() - t0))

mean squared loss at iteration    0 is 9.5217
mean squared loss at iteration  200 is 0.0708
mean squared loss at iteration  400 is 0.0082
mean squared loss at iteration  600 is 0.0011
mean squared loss at iteration  800 is 0.0002
mean squared loss at iteration 1000 is 0.0000
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[0.01446281],
       [1.006089  ],
       [1.9856374 ],
       [2.9991345 ],
       [3.992287  ]], dtype=float32)>
time took: 0.6380012035369873 seconds


**In graph mode**

In [9]:
t0 = time.time()
weights = tf.Variable(tf.random.uniform((5, 1)), dtype=tf.float32)
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        y_hat = tf.linalg.matmul(x, weights)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    gradients = tape.gradient(loss, weights)
    weights.assign_add(-0.05 * gradients)
    return loss

for iteration in range(1001):
    loss = train_step()
    if not (iteration % 200):
        print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))
        
pprint(weights)
print('time took: {} seconds'.format(time.time() - t0))

mean squared loss at iteration    0 is 10.4993
mean squared loss at iteration  200 is 0.0646
mean squared loss at iteration  400 is 0.0079
mean squared loss at iteration  600 is 0.0011
mean squared loss at iteration  800 is 0.0002
mean squared loss at iteration 1000 is 0.0000
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[0.01437845],
       [1.0059469 ],
       [1.9856571 ],
       [2.999293  ],
       [3.992368  ]], dtype=float32)>
time took: 0.27387380599975586 seconds
